# Springboard JSON Exercises
Data Science w/ Python Curriculum
Submitted by: Josh Karpen

## Import Relevant Libraries

In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize

## Import Data

In [9]:
df = pd.read_json('data/world_bank_projects.json')

## Exercise 1 - Find Top 10 Countries w/ Most Projects

In [6]:
#Check what columns are in the dataset... 
#Project_name and countryname could help answer this question
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 50 columns):
_id                         500 non-null object
approvalfy                  500 non-null int64
board_approval_month        500 non-null object
boardapprovaldate           500 non-null object
borrower                    485 non-null object
closingdate                 370 non-null object
country_namecode            500 non-null object
countrycode                 500 non-null object
countryname                 500 non-null object
countryshortname            500 non-null object
docty                       446 non-null object
envassesmentcategorycode    430 non-null object
grantamt                    500 non-null int64
ibrdcommamt                 500 non-null int64
id                          500 non-null object
idacommamt                  500 non-null int64
impagency                   472 non-null object
lendinginstr                495 non-null object
lendinginstrtype            495 non

In [45]:
#First check if project_names are actually unique
proj_counts = df['project_name'].value_counts().to_frame().reset_index().sort_values('project_name', ascending=False)
print(proj_counts.head())
print(" ")
print(proj_counts.sum())

                                                 index  project_name
0    Timor Leste Road Climate Resilience Project - ...             1
329  Integration of Children with Disabilities into...             1
342  AusAid TA for Vietnam Rural Water Supply and S...             1
341   Hunan Forest Restoration and Development Project             1
340  Additional Financing Energy Efficiency - Indus...             1
 
index           Timor Leste Road Climate Resilience Project - ...
project_name                                                  500
dtype: object


Each Project appears only once so we do not have duplicate records.

In [46]:
#Group df by countryname and get a count of project_name per country
df_country_proj = df.groupby('countryname')['project_name'].count().to_frame().sort_values('project_name', ascending=False)


The top 10 countries with the most projects are:

In [47]:
print(df_country_proj.head(10))

                                 project_name
countryname                                  
People's Republic of China                 19
Republic of Indonesia                      19
Socialist Republic of Vietnam              17
Republic of India                          16
Republic of Yemen                          13
People's Republic of Bangladesh            12
Nepal                                      12
Kingdom of Morocco                         12
Republic of Mozambique                     11
Africa                                     11


Africa is a continent, not a country, so this data might need cleaning.

## Exercise 2 - Find Top 10 Major Project Themes

In [52]:
#Inspect column mjtheme_namecode
df['mjtheme_namecode'].head()

0    [{'code': '8', 'name': 'Human development'}, {...
1    [{'code': '1', 'name': 'Economic management'},...
2    [{'code': '5', 'name': 'Trade and integration'...
3    [{'code': '7', 'name': 'Social dev/gender/incl...
4    [{'code': '5', 'name': 'Trade and integration'...
Name: mjtheme_namecode, dtype: object

This column appears to be a nested JSON with Codes and Names embedded within, so we cannot do a groupby and count the values just yet. We will need to normalize this column first using json_normalize. First we reload the dataset using json.load() because the dataframe previously created does not work with json_normalize.

In [176]:
#Reopening the dataset using json.load to get around 'string indices must be integers' error
with open('data/world_bank_projects.json') as file:
    data = json.load(file)
#pass data through json_normalize
df_themes = json_normalize(data, 'mjtheme_namecode', ['id'])
#Check whether it looks better for analysis now
print(df_themes.head())

  code                                   name       id
0    8                      Human development  P129828
1   11                                         P129828
2    1                    Economic management  P144674
3    6  Social protection and risk management  P144674
4    5                  Trade and integration  P145310


AttributeError: 'list' object has no attribute 'mjtheme_namecode'

Now do a groupby operation to view the count of theme Code and Name and sort in descending order.

In [134]:
df_themes_grouped_name = df_themes.groupby(['code', 'name'])['id'].count().to_frame().sort_values('id', ascending=False) 
print("Top 10 Themes by Code and Name")
print(df_themes_grouped_name.head(10))

Top 10 Themes by Code and Name
                                                    id
code name                                             
11   Environment and natural resources management  223
10   Rural development                             202
8    Human development                             197
2    Public sector governance                      184
6    Social protection and risk management         158
4    Financial and private sector development      130
7    Social dev/gender/inclusion                   119
5    Trade and integration                          72
9    Urban development                              47
1    Economic management                            33


This looks good but checking the output two cells higher, we can see that Code 11 does not have a Name in that output, but in the Top 10 Themes list it does have a name. This indicates that some Codes are missing Names, but only in certain rows of the data. So we are likely under-counting values.

In [135]:
df_themes_grouped = df_themes.groupby('code')['id'].count().to_frame().sort_values('id', ascending=False)
print("Top 10 Themes by Code Only")
print(df_themes_grouped.head(10))


Top 10 Themes by Code Only
       id
code     
11    250
10    216
8     210
2     199
6     168
4     146
7     130
5      77
9      50
1      38


Removing Name from the groupby lets us see the counts by Code alone. We can see that some counts are higher (Code 11 goes from 223 to 250) but the ranking is the same. However it would be nice to get the correct values and be able to see the name at the same time...

# Exercise 3 - Fill In the Missing Names

In order to create a dataframe with the blank names filled in, first we can create a dataframe with only the unique Code and Name values, without blanks.

In [158]:
#Create a dataframe with Codes and non-blank Names (only filled-in names kept).
#Remove blank names
df_themes_filled_names = df_themes[df_themes.name != '']
#Drop 'id' column
df_themes_filled_names = df_themes_filled_names.drop('id', axis=1)
#Change the index to Code
df_themes_filled_names.set_index('code', inplace=True)
#Remove duplicate rows, leaving only unique Code and Name values
df_themes_filled_names.drop_duplicates(inplace=True)

print(df_themes_filled_names.sort_values('code'))


                                              name
code                                              
1                              Economic management
10                               Rural development
11    Environment and natural resources management
2                         Public sector governance
3                                      Rule of law
4         Financial and private sector development
5                            Trade and integration
6            Social protection and risk management
7                      Social dev/gender/inclusion
8                                Human development
9                                Urban development


Next we can merge df_themes_filled_names with df_themes_grouped (the dataframe created earlier with only the Code and counts per code).


In [165]:
themes_cleaned = pd.concat([df_themes_filled_names, df_themes_grouped], axis=1, join='inner')
themes_cleaned.sort_values('id', ascending=False)

,name,id
code,,
11,Environment and natural resources management,250
10,Rural development,216
8,Human development,210
2,Public sector governance,199
6,Social protection and risk management,168
4,Financial and private sector development,146
7,Social dev/gender/inclusion,130
5,Trade and integration,77
9,Urban development,50


And now we can see the correct counts by Code and Name, with all names filled in.